<a href="https://colab.research.google.com/github/Syed2422/CIS4400/blob/main/extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob
!pip install pymongo

In [ ]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# read the config file that is json
import json
with open('config.json') as config_file:
    config = json.load(config_file)

# read information about the source
SOURCE_CONNECTION_STRING = config['SOURCE_CONNECTION_STRING']
SOURCE_CONTAINER_NAME = config['SOURCE_CONTAINER_NAME']
SOURCE_FILE_NAME = config['SOURCE_FILE_NAME']

# read information about the destination
DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config['DESTINATION_FILE_NAME']

# read information about mongodb
MONGODB_CONNECTION_STRING = config['MONGODB_CONNECTION_STRING']
MONGO_DB_NAME = config['MONGO_DB_NAME']
MONGO_COLLECTION_NAME = config['MONGO_COLLECTION_NAME']

In [ ]:
# read from source to destination azure blob

# Instantiate BlobServiceClient for the source
source_blob_service_client = BlobServiceClient.from_connection_string(SOURCE_CONNECTION_STRING)

# Get a client for the source blob to get its URL
source_blob_client = source_blob_service_client.get_blob_client(container=SOURCE_CONTAINER_NAME, blob=SOURCE_FILE_NAME)

# download the file to a specific folder
with open("downloaded_file.csv", "wb") as my_blob:
    download_stream = source_blob_client.download_blob()
    my_blob.write(download_stream.readall())

# upload to the destination
destination_blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)

# Get a client for the destination container
destination_container_client = destination_blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

# Upload the downloaded file to the destination
with open("downloaded_file.csv", "rb") as data:
    destination_blob_client = destination_container_client.upload_blob(name=DESTINATION_FILE_NAME, data=data, overwrite=True)

print(f"Downloaded file uploaded to Azure Blob '{DESTINATION_FILE_NAME}' in container '{DESTINATION_CONTAINER_NAME}'.")